# Base de Datos
### Leer base de datos

leer la base de datos "data/Base_informada" 


In [4]:
import re
import pandas as pd
from openpyxl import load_workbook

# Definir una función para leer la hoja de Excel en un DataFrame
def leer_excel_a_df(worksheet):
    data = []
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)
    
    df = pd.DataFrame(data, columns=titulos)
    return df


# Función para eliminar el contador del final del texto
def formatear_contador(texto):
    # Eliminar cualquier "-(número)" al final del texto
    texto_procesado = re.sub(r'-+\(\d+\)$', '', texto)
    return texto_procesado

In [5]:
import re

def colocar_guion_espacio_v9(texto):
    # Remove extra spaces and specific unwanted terms
    texto = texto.strip().replace("N°", "").replace(" N", "").replace(".", "").replace("EXPTE", "").replace("EXP", "").replace(":", "").replace("-", "").replace('"',"")
    
    # Specific phrases and prefixes that should trigger the special format
    triggers = [
        "RESTRICCIÓN A LA LIBERTAD", "OTRO MANDATO JUDICIAL", "ACTUACIÓN JUDICIAL", 
        "ACTUACIONES JUDICIALES", "IPP PP", "LEG CONTRAVENCIONAL", "LEGAJO FISCAL", 
        "LEGAJO AC", "LEG INVEST FISCAL", "OSL", "OSR", "OSC", "AP", "PS"
    ]
    
    # Check if text starts with any special trigger phrase or prefix
    for trigger in triggers:
        if texto.upper().startswith(trigger.upper()):
            texto = texto[len(trigger):].strip()
            
            # Extract leading number or default to "0000"
            match_number = re.match(r"(\d+)", texto)
            numero = match_number.group(0).zfill(4) if match_number else ""
            texto = texto[len(numero):].strip() if match_number else texto
            
            # Extract location (letters and/or numbers)
            match_location = re.match(r"([A-Za-z0-9]+)", texto)
            location = match_location.group(0).upper() if match_location else ""
            texto = texto[len(location):].strip() if match_location else texto
            
            # Set year and suffix
            year = "/24"
            match_suffix = re.search(r'\((\d+)\)$', texto)
            suffix = f"-({match_suffix.group(1)})" if match_suffix else "-(1)"
            
            return f"{trigger}-{numero}-{location}{year}{suffix}"
    
    # Step to identify and protect any prefix that matches a pattern like 'LL4', 'UR5', etc.
    texto = re.sub(r'\b([A-Za-z]{2,3}\d)\b', r'\1', texto)

    # Replace spaces with hyphens and insert hyphens between letters and numbers
    texto = re.sub(r'\s+', '-', texto)  
    texto = re.sub(r'([A-Za-z])(\d)', r'\1-\2', texto)
    texto = re.sub(r'(\d)([A-Za-z])', r'\1-\2', texto)

    # Restore protected prefixes by removing '' markers
    texto = re.sub(r'([A-Za-z]{2,3}\d)', r'\1', texto)

    # Remove unnecessary hyphens before slashes and format suffix
    texto = re.sub(r'-+/', '/', texto)
    if re.search(r'\(\d+\)$', texto):
        texto = re.sub(r'\((\d+)\)$', r'-(\1)', texto)
    else:
        texto += "-(1)"
    
    # Remove multiple consecutive hyphens
    texto = re.sub(r'-+', '-', texto)

    return texto



In [6]:
# Cargar el archivo Excel y la hoja requerida
nombre_base = "data/BASE_JORGE.xlsx"
wb = load_workbook(nombre_base)
hoja = wb['GEOG. PROCEDIMIENTO']

# Crear un DataFrame a partir de los datos de la hoja
df_base_informada = leer_excel_a_df(hoja)

# Aplicar las funciones de procesamiento a las columnas necesarias
df_base_informada["ID_PROCEDIMIENTO"] = df_base_informada["ID_PROCEDIMIENTO"].apply(colocar_guion_espacio_v9)
df_base_informada["ID_OPERATIVO"] = df_base_informada["ID_PROCEDIMIENTO"].apply(formatear_contador)

# Escribir los datos del DataFrame en la hoja 'GEOG. PROCEDIMIENTO' desde la fila 4
for row_num, row in enumerate(df_base_informada.itertuples(index=False), start=4):
    for col_num, value in enumerate(row, start=2):  # Comienza desde la columna B (número 2)
        hoja.cell(row=row_num, column=col_num).value = value

# Guardar los cambios en el archivo original o en un nuevo archivo
wb.save("data/BASE_PROCESADO.xlsx")

print("Los cambios han sido guardados correctamente.")

Los cambios han sido guardados correctamente.
